## Extração de recursos com HOG - Histogram of Oriented gradients
***
### O que é um Feature Descriptor?
***

Um Feature Descriptor (descritor  de recursos  ou  descritor  de  características)  é  uma representação  de  uma  imagem  ou  um  patch  (parte)  de  imagem  que  simplifica  a  imagem, extraindo informações úteis e eliminando informações estranhas.

Normalmente, um descritor de recursos converte uma imagem de tamanho “largura x altura  x  3 (canais)”  para  um  vetor  de  característica/matriz  de  comprimento  N.  No  caso  do descritor de característica HOG, a imagem de entrada é de tamanho 64 x 128 x 3 e o vetor de característica de saída é de comprimento 3780.

Tenha em mente que o descritor HOG pode ser calculado para outros tamanhos, mas aqui  usaremos  os  números  apresentados  no  documento  original  para  que  você  possa facilmente entender o conceito com um exemplo concreto.

Mas como definir o que é "útil" e o que é "estranho" na imagem? Para definir "útil", precisamos saber o que é "útil" para qual atividade. Claramente, o vetor de características não é útil para visualizar a imagem. Mas é muito útil para tarefas como reconhecimento de imagens e  detecção  de  objetos.  O  vetor  de  características  produzido  por  esses  algoritmos, quando alimentado em algoritmos de classificação de imagem como o Support Vector Machine (SVM), produz bons resultados.

Mas, que tipos de "recursos" são úteis para tarefas de classificação? Vamos discutir este ponto  usando  um  exemplo.  Suponha  que  queremos  construir  um  detector  de  objetos  que detecte  botões  de  camisas  e  casacos.  Um  botão  é  circular  (pode  parecer  elíptico  em  uma imagem) e geralmente possui alguns orifícios para costura. Você pode executar um detector de borda na imagem de um botão, e facilmente dizer se é um botão simplesmente olhando apenas a imagem da borda. Nesse caso, as informações de borda são "úteis" e as informações de cores não são. Além disso, os recursos também precisam ter poder discriminativo. Por exemplo, os bons recursos extraídos de uma imagem devem ser capazes de dizer a diferença entre botões e outros objetos circulares, como moedas e pneus de carro.

No descritor do recurso HOG, a distribuição (histograma) das direções de gradientes (gradientes orientados) é usada como característica. Os gradientes (derivadas de x e y) de uma imagem são úteis porque a magnitude dos gradientes é grande em torno de bordas e cantos (regiões de mudanças abruptas de intensidade) e sabemos que bordas e cantos tem muito mais informações sobre a forma do objeto do que as regiões planas.

***
### Como calcular o histograma de gradientes orientados?
***

Vejamos os detalhes do cálculo do descritor do recurso HOG. Para ilustrar cada passo, usaremos um patch de uma imagem.

***
#### Passo 1: pré-processamento
***

Como mencionado anteriormente, o descritor do recurso HOG usado para detecção de objetos é calculado em um patch de uma imagem de 64×128. Claro, uma imagem pode ser de qualquer tamanho. Normalmente, os patches em escalas múltiplas são analisados em muitos locais  de  imagens.  A  única  restrição  é  que  os  patches  em  análise  tenham  uma  relação  de aspecto (aspect ratio) fixa. No nosso caso, os patches precisam ter uma relação de aspecto de 1:2. Por exemplo, eles podem ser 100×200, 128×256 ou 1000×2000, mas não 101×205.

Para  ilustrar  este  ponto, considere uma  grande  imagem  de  tamanho  720×475. Selecionamos um patch de tamanho 100×200 para calcular nosso descritor de recurso HOG. Este patch é cortado de uma imagem e redimensionado para 64×128. Agora, estamos prontos para calcular o descritor HOG para este patch de imagem.

![img](https://user-images.githubusercontent.com/14116020/64903779-71d89000-d695-11e9-9080-5a4593a0e8ec.png)

O artigo de Dalal e Triggs também menciona a correção de gama como  um  passo  de  pré-processamento,  mas  os  ganhos  de  desempenho  são  menores  e, portanto, estamos ignorando o passo.

***
#### Passo 2: Calcule as imagens de gradiente
***

Para calcular um descritor HOG, precisamos primeiro calcular os gradientes horizontal e vertical.  Afinal,  queremos  calcular  o  histograma  de  gradientes.  Isso  é  facilmente  alcançado através da filtragem da imagem com os seguintes kernels.

![img](https://user-images.githubusercontent.com/14116020/64903791-c54ade00-d695-11e9-9eb6-dcd76fe6eb2b.png)

Também podemos obter os mesmos resultados, usando o operador Sobel no OpenCV com o tamanho 1 do kernel.

```py
# Leitura da imagem
img = cv2.imread("bolt.png")
img = np.float32(img) / 255.0

# Calcula os gradientes
gx = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=1)
gy = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=1)
```

Em  seguida,  podemos  encontrar  a  magnitude  e  a  direção  do  gradiente  usando  a seguinte fórmula:

$$g = \sqrt{g_x^2 + g_y^2}$$

$$\theta = arctan\frac{g_y}{g_x}$$

Se você estiver usando OpenCV, o cálculo pode ser feito usando a função:

```py
mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
```

A figura abaixo mostra os gradientes.

![img](https://user-images.githubusercontent.com/14116020/64903816-689bf300-d696-11e9-8ff6-984d8270c4be.png)

Observe  como o  gradiente  x  dispara  em  linhas  verticais  e o gradiente  y  em  linhas horizontais.  A  magnitude  dos gradientes  dispara sempre  que há uma  forte  mudança  de intensidade. Nenhum deles dispara quando a região é suave.

A imagem de gradiente removeu uma grande quantidade de informações não essenciais (por exemplo, fundo colorido constante), mas destacou os contornos destacados. Em outras palavras, você pode olhar a imagem de gradiente e ainda assim dizer facilmente que há uma pessoa na imagem.

Em cada pixel, o gradiente tem uma magnitude e uma direção. Para imagens em cores, os gradientes dos três canais são avaliados (como mostrado na figura acima). A magnitude do gradiente em um pixel é o máximo da magnitude dos gradientes dos três canais e o ângulo é o ângulo correspondente ao gradiente máximo.

***
#### Passo 3: Calcule o Histograma de Gradientes em células 8x8
***

Nesta etapa, a imagem é dividida em células 8×8 e um histograma de gradientes é calculado para cada célula 8×8.

Vamos entender  porque  dividimos  a  imagem  em células 8×8. Um  dos  motivos importantes para usar um descritor de recurso para descrever um patch de uma imagem é que ele fornece uma representação compacta. Um patch de imagem de 8×8 contém 8x8x3 = 192 pixels de valores. O gradiente deste patch contém 2 valores (magnitude e direção) por pixel que somam até 8x8x2 = 128 números. Ao final desta seção, veremos como esses 128 números são representados usando um histograma de 9 binários que podem ser armazenados como uma matriz de 9 números. Não só a representação é mais compacta, o cálculo de um histograma sobre um patch torna esta representação mais robusta ao ruído. Os graus individuais podem ter ruído, mas um histograma sobre um patch de 8×8 torna a representação muito menos sensível ao ruído.

Mas por que o patch de 8×8? Por que não 32×32? É uma escolha de design informada pela escala de recursos que procuramos. HOG foi utilizado inicialmente para a detecção de pedestres. Células 8×8 em uma foto de um pedestre em escala para 64×128 são grandes o suficiente para capturar características interessantes (por exemplo, o rosto, o topo da cabeça, etc.).

O  histograma  é  essencialmente um  vetor  (ou  uma  matriz)  de  9  caixas  (números) correspondentes aos ângulos 0, 20, 40, 60 ... 160. Veja na figura abaixo, um patch de 8×8 na imagem e veja como os gradientes se parecem.

![img](https://user-images.githubusercontent.com/14116020/64903849-1a3b2400-d697-11e9-9672-76220fe000a0.png)

A imagem no centro é muito informativa. Ela mostra o patch da imagem sobreposta com setas mostrando o gradiente - a seta mostra a direção do gradiente e seu comprimento mostra a magnitude. Observe como a direção das setas aponta para a direção da mudança de intensidade e a magnitude mostra quão grande é a diferença.

À  direita,  vemos  os  números  em formato bruto  que  representam  os  gradientes  nas células 8×8 com uma pequena diferença - os ângulos estão entre 0 e 180 graus em vez de 0 a 360  graus.  Estes  são  chamados  de  gradientes  "não  assinados"  porque  um  gradiente e  seu negativo  são  representados  pelos  mesmos  números.  Em  outras  palavras,  uma  flecha  de gradiente e seu oposto a 180 graus são consideradas as mesmas. Mas, por que não usar os 0 a 360  graus?  Empiricamente,  foi  demonstrado  que  os  gradientes  não  assinados  funcionam melhor do que os gradientes assinados para a detecção de pedestres. Algumas implementações do HOG permitirão especificar se você deseja usar gradientes assinados.

O próximo passo é criar um histograma de gradientes nessas células 8×8. O histograma contém 9 caixas correspondentes aos ângulos 0, 20, 40 ... 160.

A  figura  a  seguir  ilustra  o  processo.  Estamos  olhando  a  magnitude  e  direção  do gradiente do mesmo patch de 8×8 como na figura anterior. Um compartimento é selecionado com base na direção, e a votação (o valor que se encontra no compartimento) é selecionada com  base  na  magnitude.  Vamos  primeiro  focar  no  pixel  cercado  em  azul.  Tem  um  ângulo (direção) de 80 graus e magnitude de 2. Então, ele adiciona 2 na 5ª caixa. O gradiente no pixel circundado usando vermelho tem um ângulo de 10 graus e magnitude de 4. Uma vez que 10 graus estão a meio caminho entre 0 e 20, o voto pelo pixel se separa uniformemente nas duas caixas.

![img](https://user-images.githubusercontent.com/14116020/64903887-a8afa580-d697-11e9-86a4-8eebb6f2c0bc.png)

Há mais um detalhe. Se o ângulo for superior a 160 graus, entre 160 e 180, sabemos que o  ângulo  se  envolve  ao  fazer  0  e  180  equivalentes.  Assim,  no  exemplo  abaixo,  o  pixel  com ângulo  165  graus  contribui  proporcionalmente  para  o  compartimento  de  0  graus  e  o compartimento de 160 graus.

![img](https://user-images.githubusercontent.com/14116020/64903900-ca109180-d697-11e9-9b9b-ab5557bafc26.png)

As  contribuições  de  todos  os  pixels  das  células  8×8  são  adicionadas  para  criar  o histograma de 9 bin. Para o patch acima, parece assim:

![img](https://user-images.githubusercontent.com/14116020/64903905-e3194280-d697-11e9-9faa-eda9b13840ff.png)

Em nossa representação, o eixo y é de 0 graus. Você pode ver que o histograma tem muito peso perto de 0 e 180 graus, o que é apenas outra maneira de dizer que no patch os gradientes estão apontando para cima ou para baixo.

***
#### Passo 4: 16×16 Normalização do bloco
***

No  passo  anterior,  criamos  um  histograma  baseado  no  gradiente  da  imagem.  Os gradientes de uma imagem são sensíveis à iluminação geral. Se você tornar a imagem mais escura dividindo todos os valores de pixel em 2, a magnitude do gradiente mudará pela metade e, portanto, os valores do histograma mudarão pela metade. Idealmente, queremos que nosso descritor seja independente das variações de iluminação. Em outras palavras, gostaríamos de "normalizar" o histograma para que eles não sejam afetados por variações de iluminação.

Antes  de  explicar  como  o  histograma  é  normalizado,  vejamos  como um  vetor  de comprimento 3 é normalizado.

![img](https://user-images.githubusercontent.com/14116020/64903925-2c699200-d698-11e9-9565-aa52003da66e.png)

Digamos que temos um vetor de cores RGB [128, 64, 32]. O comprimento deste vetor é $\sqrt{128^2 + 64^2 + 32^2} = 146.64$.  Isso  também  é  chamado  de  norma  L2  do  vetor.  Dividir  cada elemento  deste  vetor  por  146,64  nos  dá  um  vetor  normalizado  [0.87, 0.43,  0.22].  Agora considere outro vetor em que os elementos são duas vezes o valor do primeiro vetor 2 x [128, 64, 32] = [256, 128, 64]. Você pode resolver isso mesmo para ver que a normalização [256, 128, 64] resultará em [0.87, 0.43, 0.22], que é o mesmo que a versão normalizada do vetor RGB original. Você pode ver que a normalização de um vetor remove a escala.

Agora que sabemos como normalizar um vetor, você pode estar tentado a pensar que ao  calcular  o  HOG  você  pode  simplesmente  normalizar  o  histograma  de  9×1  da  mesma maneira que normalizamos o vetor 3×1 acima. Não é uma má ideia, mas uma ideia melhor é normalizar em um bloco de tamanho maior de 16×16. Um bloco de 16×16 tem 4 histogramas que podem ser concatenados para formar um vetor de 36x1 elemento e pode ser normalizado exatamente como um vetor de 3×1 é normalizado. A janela é então movida por 8 pixels e um vetor normalizado de 36×1 é calculado sobre esta janela e o processo é repetido.

***
#### Passo 5: Calcule o vetor de características HOG
***

Para calcular o vetor de características finais para todo o patch de imagem, os vetores 36×1 são concatenados em um vetor gigante. Qual é o tamanho desse vetor? Vamos calcular.

Quantas  posições  dos blocos 16×16  blocos?  Existem  7  posições  horizontais  e  15 verticais, totalizando 7 x 15 = 105 posições.Cada bloco 16×16 é representado por um vetor de 36×1. Então, quando os concatenamos em um único vetor, obtemos um vetor dimensional de 36 × 105 = 3780.

O  descritor  HOG  de  um  patch  de  imagem  geralmente  é  visualizado  por  traçar  os histogramas normalizados 9×1 nas células 8×8. Veja a imagem abaixo. Você notará que a direção  dominante  do  histograma  captura  a  forma  da  pessoa,  especialmente  em  torno  do tronco e das pernas. Infelizmente, não existe uma maneira fácil de visualizar o descritor HOG no OpenCV.

![img](https://user-images.githubusercontent.com/14116020/64903980-dba66900-d698-11e9-8d7b-0c6167c9538d.png)